In [2]:
# ==============================
# 🔧 Instalação de pacotes
# ==============================
!apt update -y
!apt install -y tesseract-ocr poppler-utils
!pip install --upgrade pytesseract Pillow pdf2image PyMuPDF easyocr gspread oauth2client \
    google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2 \
    gspread-formatting opencv-python-headless numpy

# ==============================
# 📂 Montar Google Drive
# ==============================
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# ==============================
# 📦 Importações
# ==============================
import os
import re
import fitz  # PyMuPDF
from PIL import Image
import pytesseract
import easyocr
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pdf2image import convert_from_path
from gspread_formatting import set_row_height, set_column_width, format_cell_range, CellFormat, TextFormat, set_frozen

# ==============================
# 🔐 Autenticação Google
# ==============================
scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

json_path = '/content/drive/MyDrive/Colab Notebooks/caetanodecampos-a97889ef1071.json'
if not os.path.exists(json_path):
    raise FileNotFoundError(f"Arquivo JSON não encontrado: {json_path}")

creds = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
client = gspread.authorize(creds)

spreadsheet_id = '1nysE4mZ8xzFzEE5IibUaU65qaG3z6ftlSYJtWuijVOg'
spreadsheet = client.open_by_key(spreadsheet_id)
sheet = spreadsheet.worksheet('sheet1')

# ==============================
# 🖼️ Pasta de arquivos
# ==============================
image_folder = '/content/drive/MyDrive/Colab Notebooks/contracapas'

# ==============================
# 🧠 Função OCR
# ==============================
reader = easyocr.Reader(['pt'], gpu=True)

def extrair_dados(texto):
    titulo = autor = ano = editora = 'Não encontrado'
    padrao_titulo = r'^(.*?)\n'
    padrao_autor = r'(?i)Azevedo,\s*(.*?)\n|Autor[:\s]*(.*?)(?:\n|$)'
    padrao_ano = r'(\d{4})'
    padrao_editora = r'(?i)Editora[:\s]*(.*?)(?:\n|$)'

    match_titulo = re.search(padrao_titulo, texto)
    if match_titulo:
        titulo = match_titulo.group(1).strip()

    match_autor = re.search(padrao_autor, texto)
    if match_autor:
        autor = (match_autor.group(1) or match_autor.group(2)).strip()

    match_ano = re.search(padrao_ano, texto)
    if match_ano:
        ano = match_ano.group(1).strip()

    match_editora = re.search(padrao_editora, texto)
    if match_editora:
        editora = match_editora.group(1).strip()

    return titulo, autor, ano, editora

# ==============================
# 🔁 Processar arquivos
# ==============================
def arquivos_ordenados(pasta):
    return sorted(os.listdir(pasta), key=lambda x: [int(t) if t.isdigit() else t for t in re.split(r'(\d+)', x)])

arquivos = arquivos_ordenados(image_folder)

# Verificar duplicatas na planilha
dados_existentes = [row[4] for row in sheet.get_all_values()[1:]]

for file_name in arquivos:
    if file_name in dados_existentes:
        print(f"⚠️ Pulando {file_name} (já existe na planilha)")
        continue

    file_path = os.path.join(image_folder, file_name)
    texto_extraido = ""

    # PDF
    if file_name.lower().endswith('.pdf'):
        pages = convert_from_path(file_path)
        for page in pages:
            texto_extraido += ' ' + ' '.join([line[1] for line in reader.readtext(np.array(page))])
    # Imagem
    elif file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img = Image.open(file_path)
        texto_extraido = ' '.join([line[1] for line in reader.readtext(np.array(img))])
    else:
        print(f"⚠️ Formato não suportado: {file_name}")
        continue

    titulo, autor, ano, editora = extrair_dados(texto_extraido)
    print(f"📚 {file_name} -> {titulo} | {autor} | {ano} | {editora}")

    sheet.append_row([titulo, autor, ano, editora, file_name])

# ==============================
# 🎨 Formatação da planilha
# ==============================
# Congelar primeira linha
set_frozen(sheet, rows=1)

# Largura das colunas
set_column_width(sheet, 'A', 300)
set_column_width(sheet, 'B', 200)
set_column_width(sheet, 'C', 100)
set_column_width(sheet, 'D', 200)
set_column_width(sheet, 'E', 150)

# Altura da primeira linha
set_row_height(sheet, '1:1', 30)

# Cabeçalho negrito e centralizado
header_format = CellFormat(
    textFormat=TextFormat(bold=True),
    horizontalAlignment='CENTER'
)
format_cell_range(sheet, 'A1:E1', header_format)


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
4 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as rep

⚠️ Pulando 1.1.capa.png (já existe na planilha)
⚠️ Pulando 1.1.contra.png (já existe na planilha)
⚠️ Pulando 1.2.contra.png (já existe na planilha)
⚠️ Pulando 1.2capa.png (já existe na planilha)
⚠️ Pulando 1.3.capa.png (já existe na planilha)
⚠️ Pulando 1.3.contra.png (já existe na planilha)
⚠️ Pulando 1.4.capa.png (já existe na planilha)
⚠️ Pulando 1.4.contra.png (já existe na planilha)
⚠️ Pulando 1.5.capa.jpeg (já existe na planilha)
⚠️ Pulando 1.5.contra.jpg (já existe na planilha)
⚠️ Pulando 1.6.capa.png (já existe na planilha)
⚠️ Pulando 1.6.contra.png (já existe na planilha)
⚠️ Pulando 1.7..png (já existe na planilha)
⚠️ Pulando 1.7.contra.png (já existe na planilha)
⚠️ Pulando 1.8.capa.png (já existe na planilha)
⚠️ Pulando 1.8.contra.png (já existe na planilha)
⚠️ Pulando 1.capa.png (já existe na planilha)
⚠️ Pulando 1.contra.png (já existe na planilha)
⚠️ Pulando 5 (1).capa.pdf (já existe na planilha)
⚠️ Pulando 5.1 (1).capa.pdf (já existe na planilha)
⚠️ Pulando 5.1.capa.pdf 

{'spreadsheetId': '1nysE4mZ8xzFzEE5IibUaU65qaG3z6ftlSYJtWuijVOg',
 'replies': [{}]}